# Estimation

The first section of this notebook will go through our thought process, steps, and execution of demand and supply estimation.

## Demand-Side

Assume for simplicity that $a$ and $b$ are known. In addition, set a consumer population $P$ and, using observed quantities, determine market shares $s_0, s_1, \ldots, s_N$ for products $j = 1,\ldots,N$ and outside option $j=0$.

In the vertical model, market shares are determined by cutoffs between neighboring goods, where ordering is based on quality $\delta_j$. Quality is unknown initially, but sorting by price should yield the same ordering (since no one will buy a product with a higher price tag and a lower quality than another product). 

Thus, assuming products are ordered by increasing price, the market share of product $j$ is given by the probability of a consumer's taste falling between the points at which $\nu\delta_j - p_j = \nu\delta_{j+1} - p_{j+1}$ and $\nu\delta_j - p_j = \nu\delta_{j-1} - p_{j-1}$. These cutoff points, which we call $\Delta_j$ and $\Delta_{j+1}$, are given by $$\Delta_j = \frac{p_{j} - p_{j-1}}{\delta_{j} - \delta_{j-1}}$$

Then, given the CDF $F(\cdot)$ of $\nu \sim \text{Unif}[a,b]$, all $\Delta$'s must satisfy the market share equations

$$s_j = F(\Delta_{j+1}) - F(\Delta_j)$$

where $\Delta_0 = -\infty$ and $\Delta_{N+1} = \infty$. Then, $\Delta_j$ can be recursively determined by

$$\Delta_j = F^{-1}(F(\Delta_{j+1}) - s_j)$$

starting from $\Delta_N = (b-a)(1-s_N)$. By expressing $\delta_j$ in terms of $\Delta_j$, we can recursively recover each $\delta_j$ using the initial conditions $\delta_0 = 0$ and $p_0 = 0$:

$$\delta_j = \delta_{j-1} + \frac{p_j - p_{j-1}}{\Delta_j}$$

As a result, unobservable quality enters linearly and one can run a regression to determine $\beta$ from $\xi_j = \delta_j - x_j'\beta$.

The nature of this regression depends on our assumptions on $\xi_j$. If $\mathbb{E}[\xi_j|x_j, p]$ is assumed, then OLS suffices. We cannot make such an assumption here, so we use some characteristics of competing products, $x_{-j}$, as instruments and run IV GMM to estimate $\beta$.

Due to the small number of unique values observed for horsepower and air conditioning, the stats program encountered a singular matrix error when these characteristics were part of the instruments. Weight admits a greater diversity of values, so our instrument is a $N-1$ dimensional vector of the weights of competitor's cars.

In our estimation procedure below, we assume that $a = 0$ and $b = 1$. Additionally, Bresnahan's observed total quantity sold is similar to ours, so we would expect the consumer base to be approximately the same (so that $P = 20000000$).

In [382]:
import pandas as pd
import numpy as np
import statsmodels.sandbox.regression.gmm as stats

autodata = pd.read_csv('./sample_data/autodata.csv')

N = len(autodata)

# Bresnahan 1987 finds a similar volume of cars sold
# compared to our data - thus his consumer base size is
# reasonable for our usage
P = 20000000 

a,b = 0, 1 # taste distribution parameters
Delta = np.zeros((N, 1)) # cutoffs
delta = np.zeros((N, 1)) # mean utility

autodata = autodata.sort_values(by=['Price($1000) (list price)']).reset_index(drop=True) # order by "quality"
shares = autodata['Quantity'] / P
prices = autodata['Price($1000) (list price)']
x = autodata[['Air Conditioning', 'Weight of Car', 'Horsepower']]


# Calculate cutoffs
Delta[N-1] = (b-a)*(1-shares[N-1])

for j in range(N-2, -1, -1):
  Delta[j] = (b-a)*(max(0, min(Delta[j+1], 1)) - shares[j])

# Recover mean utility
delta[0] = prices[0] / Delta[0]

for j in range(1, N):
  delta[j] = delta[j-1] + (prices[j] - prices[j-1]) / Delta[j]

instruments = np.zeros((N, N-1))

for i in range(N):
  d = autodata.drop(i)
  instruments[i,:] = np.array(d[['Weight of Car']]).reshape((N-1,))

iv = stats.LinearIVGMM(delta, x, instruments)
reg = iv.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             LinearIVGMM Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                        2658.
Model:                    LinearIVGMM   Prob (Hansen J):                  0.00
Method:                           GMM                                         
Date:                Tue, 07 Jun 2022                                         
Time:                        21:32:22                                         
No. Observations:                 131                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Air Conditioning  4650.5815    225.005     20.669      0.000    4209.580    5091.583
Weight of Car        0.0530      0.061      0.873      0.383      -0.066       0.172
Horsepower         154.6925      1.355    114.195      0.000     152.037     157.348
====================================================================================
"""

## Supply Side

From equation (29) in Berry (1994), the first order condition under a (product-level) Nash equilibrium takes the form

$$
\begin{align*}
p_j &= x_j\gamma + \lambda q_j + \frac{s_j}{  \partial s_j/ \partial \delta _j} + \omega _j,
\end{align*}
$$



In [383]:
autodata = autodata.rename(columns={"Price($1000) (list price)": "Price", "Firm ID (there is a different number for each firm)": "Firm ID"})


#### (i) Marginal cost pricing

If price is set at marginal cost, 
$$
p_j = mc_j = x_j\gamma + \lambda q_j +  \omega _j
$$

Since we observe price, we can estimate the parameters directly. Here, $\omega$ is uncorrelated with $x$ but possibly correlated with $q$. However, we can use the characteristics of other products, $x_{-j}$, as an instrument for $q_j$, to estimate $\gamma$ and $\lambda$ via 2SLS.

In [384]:
modivp2 = stats.LinearIVGMM(prices, np.column_stack((x, autodata['Quantity'])), instrument= instruments).fit()
modivp2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 LinearIVGMM Results                                 
=====================================================================================
Dep. Variable:     Price($1000) (list price)   Hansen J:                        2135.
Model:                           LinearIVGMM   Prob (Hansen J):                  0.00
Method:                                  GMM                                         
Date:                       Tue, 07 Jun 2022                                         
Time:                               21:32:26                                         
No. Observations:                        131                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1          8604.8131    114.623     75.071      0.000    8380.157    8829.469
x2            -1.1405      0.124     -9.168      0.000      -1.384      -0.897
x3           134.1064      3.166     42.353      0.000     127.900     140.312
x4            -0.0045      0.001     -8.375      0.000      -0.006      -0.003
==============================================================================
"""

### (ii-iii) Nash pricing and perfect collution

Following Bresnahan (1987), the pricing behavior under collusion is 

$$
0 = s_j + (p_j-mc_j)\frac{\partial s_j}{\partial p _j} + H_{j, j+1} (p_{j+1}-mc_{j+1})\frac{\partial s_{j+1}}{\partial p _j} + H_{j, j-1} (p_{j-1}-mc_{j-1})\frac{\partial s_{j-1}}{\partial p _j} 
$$

Here, $H_{j, k}$ is the indicator for product $j$ and $k$ jointly maximizing profit. Under the assumption of multi-product Nash pricing, this indicator turns on if the two products are made by the same firm. Under perfect collusion, the indicator equals one for all product pairs.

The price elasticities under the vertical model is

$$
\begin{align*}

\frac{\partial s_j}{\partial p _j}& = -[\frac{f(\Delta_{j+1})}{\delta_{j+1}-\delta_{j}} + \frac{f(\Delta_{j})}{\delta_{j}-\delta_{j-1}}]\\

\end{align*}
$$
And since $F \sim \text{Unif}[0,1]$ by assumption, we can write 

$$
\begin{align*}

\frac{\partial s_j}{\partial p_j}& = -[\frac{1}{\delta_{j+1}-\delta_{j}} + \frac{1}{\delta_{j}-\delta_{j-1}}]\\
\frac{\partial s_{j-1}}{\partial p_j}& = \frac{1}{\delta_{j}-\delta_{j-1}}\\
\frac{\partial s_{j+1}}{\partial p _j}& = \frac{1}{\delta_{j+1}-\delta_{j}} \\

\end{align*}
$$

Note: cars with the same price in the data will have the same mean utility and an infinite price elasticity. We add a small pertubation to the $\delta$ term to get around this.

In [392]:
for i in range(len(delta)-1):
    if delta[i] == delta[i+1]:
        delta[i] -= 0.001
delta_0 = np.concatenate(([[0]],delta,[[np.inf]]))
dsdp = [-(1/(delta_0[j+1]-delta_0[j]) + 1/(delta_0[j]-delta_0[j-1])) for j in range(1,N+1)]
dsmdp = [1/(delta_0[j]-delta_0[j-1]) for j in range(1,N+1)]
dspdp = [1/(delta_0[j+1]-delta_0[j])for j in range(1,N+1)]

Hmat = np.array([[autodata['Firm ID'][i]==autodata['Firm ID'][j] for i in range(N)] for j in range(N)], np.int32)

From the FOC we can get the following equation, where $\tilde{\omega}_j$ is a transformation of $(\omega_j, \omega_{j+1}, \omega_{j-1})$ that is correlated with the quantities. We estimate by 2SLS $\gamma$ and $\lambda$, again with $x_{-j}$ as instruments
$$
% \frac{\partial s_j}{\partial p _j}p_j = \frac{\partial s_j}{\partial p _j}(x_j\gamma + \lambda q_j) - s_j - H_{j, j+1}(p_{j+1}-x_{j+1}\gamma - \lambda q_{j+1})\frac{\partial s_{j+1}}{\partial p _j} - H_{j, j-1}(p_{j-1}-x_{j-1}\gamma - \lambda q_{j-1})\frac{\partial s_{j-1}}{\partial p _j}\\
\frac{\partial s_j}{\partial p _j}p_j +s_j + H_{j, j+1}p_{j+1}\frac{\partial s_{j+1}}{\partial p _j} + H_{j, j-1}p_{j-1}\frac{\partial s_{j-1}}{\partial p _j}= \gamma[\frac{\partial s_j}{\partial p _j}x_j + H_{j, j+1}x_{j+1}\frac{\partial s_{j+1}}{\partial p _j} + H_{j, j-1}x_{j-1}\frac{\partial s_{j-1}}{\partial p _j}]  + \lambda[\frac{\partial s_j}{\partial p _j}q_j + H_{j, j+1}q_{j+1}\frac{\partial s_{j+1}}{\partial p _j} + H_{j, j-1}q_{j-1}\frac{\partial s_{j-1}}{\partial p _j}] + \tilde{\omega}_j

$$

In [388]:
xmat = np.array(x)
quantities = np.array(autodata['Quantity'])
lhs = np.zeros(N)
gammaterm = np.zeros((N,3))
lambdaterm = np.zeros(N)
for j in range(N): 
    if j==0:
        lhs[j] = dsdp[j]*prices[j] + Hmat[j,j+1]*prices[j+1]*dspdp[j] 
        gammaterm[j,:] = dsdp[j]*xmat[j,:] + Hmat[j,j+1]*xmat[j+1,:]*dspdp[j] 
        lambdaterm[j] = dsdp[j]*quantities[j] + Hmat[j,j+1]*quantities[j+1]*dspdp[j]
    elif j==N-1:
        lhs[j] = dsdp[j]*prices[j] +  Hmat[j,j-1]*prices[j-1]*dsmdp[j]
        gammaterm[j,:] = dsdp[j]*xmat[j,:] +  Hmat[j,j-1]*xmat[j-1,:]*dsmdp[j]
        lambdaterm[j] = dsdp[j]*quantities[j] + Hmat[j,j-1]*quantities[j-1]*dsmdp[j]
    else:
        lhs[j] = dsdp[j]*prices[j] + Hmat[j,j+1]*prices[j+1]*dspdp[j] +  Hmat[j,j-1]*prices[j-1]*dsmdp[j]
        gammaterm[j,:] = dsdp[j]*xmat[j,:] + Hmat[j,j+1]*xmat[j+1,:]*dspdp[j] +  Hmat[j,j-1]*xmat[j-1,:]*dsmdp[j]
        lambdaterm[j] = dsdp[j]*quantities[j] + Hmat[j,j+1]*quantities[j+1]*dspdp[j] +  Hmat[j,j-1]*quantities[j-1]*dsmdp[j]
        


In [389]:
modn = stats.LinearIVGMM(lhs, np.column_stack((gammaterm, lambdaterm)), instrument=instruments).fit()
modn.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             LinearIVGMM Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                        94.49
Model:                    LinearIVGMM   Prob (Hansen J):                 0.984
Method:                           GMM                                         
Date:                Tue, 07 Jun 2022                                         
Time:                        21:32:32                                         
No. Observations:                 131                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1          9634.0203    125.500     76.765      0.000    9388.045    9879.996
x2             2.8782      0.039     73.614      0.000       2.802       2.955
x3            24.3012      0.874     27.798      0.000      22.588      26.015
x4             0.0008        nan        nan        nan         nan         nan
==============================================================================
"""

Now we turn on all the $H_{i,j}$ indicators for the perfect collusion model. 

In [393]:
Hmat = np.ones((N,N))

for j in range(N): 
    if j==0:
        lhs[j] = dsdp[j]*prices[j] + Hmat[j,j+1]*prices[j+1]*dspdp[j] 
        gammaterm[j,:] = dsdp[j]*xmat[j,:] + Hmat[j,j+1]*xmat[j+1,:]*dspdp[j] 
        lambdaterm[j] = dsdp[j]*quantities[j] + Hmat[j,j+1]*quantities[j+1]*dspdp[j]
    elif j==N-1:
        lhs[j] = dsdp[j]*prices[j] +  Hmat[j,j-1]*prices[j-1]*dsmdp[j]
        gammaterm[j,:] = dsdp[j]*xmat[j,:] +  Hmat[j,j-1]*xmat[j-1,:]*dsmdp[j]
        lambdaterm[j] = dsdp[j]*quantities[j] + Hmat[j,j-1]*quantities[j-1]*dsmdp[j]
    else:
        lhs[j] = dsdp[j]*prices[j] + Hmat[j,j+1]*prices[j+1]*dspdp[j] +  Hmat[j,j-1]*prices[j-1]*dsmdp[j]
        gammaterm[j,:] = dsdp[j]*xmat[j,:] + Hmat[j,j+1]*xmat[j+1,:]*dspdp[j] +  Hmat[j,j-1]*xmat[j-1,:]*dsmdp[j]
        lambdaterm[j] = dsdp[j]*quantities[j] + Hmat[j,j+1]*quantities[j+1]*dspdp[j] +  Hmat[j,j-1]*quantities[j-1]*dsmdp[j]
        


In [394]:
modc = stats.LinearIVGMM(lhs, np.column_stack((gammaterm, lambdaterm)), instrument=instruments).fit()
modc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             LinearIVGMM Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                        270.6
Model:                    LinearIVGMM   Prob (Hansen J):              1.38e-12
Method:                           GMM                                         
Date:                Tue, 07 Jun 2022                                         
Time:                        21:38:33                                         
No. Observations:                 131                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0277      0.005      5.694      0.000       0.018       0.037
x2         -1.651e-05   7.99e-06     -2.066      0.039   -3.22e-05   -8.45e-07
x3             0.0009      0.000      8.822      0.000       0.001       0.001
x4           3.48e-08   1.17e-08      2.978      0.003    1.19e-08    5.77e-08
==============================================================================
"""

## (3) Evaluting model fit

Bresnahan (1987) uses 